# Chapter 10: Introduction to Artificial Neura Networks

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

In [ ]:
iris = load_iris()
X = iris.data[:, (2, 3)]
y = (iris.target == 0).astype(np.int)

In [ ]:
per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

In [ ]:
y_pred = per_clf.predict([[2, 0.5]])

Perceptron的缺点：直接输出分类，不支持概率

## Multi-Layer Perceptron

In [ ]:
# Setup mnist data set
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

In [ ]:
feature_columns = [tf.feature_column.numeric_column('x', shape=[28, 28])]

In [ ]:
# define DNNClassifier
dnn_clf = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir="/Users/chance/machine_learning/hands_on_machine_learning/tmp/dnn_mnist_model"
)

In [ ]:
def input(dataset):
    return dataset.images, dataset.labels.astype(np.int32) #这个int32转换是必要的

In [ ]:
# 封装了一个input函数，但需要定义input，貌似很麻烦。。。
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.train)[0]},
    y=input(mnist.train)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

In [ ]:
# Training
dnn_clf.train(input_fn=train_input_fn, steps=100000)

In [ ]:
# perpare test input
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': input(mnist.test)[0]},
    y=input(mnist.test)[1],
    num_epochs=1,
    shuffle=False,
)

In [ ]:
evaluate_result = dnn_clf.evaluate(input_fn=test_input_fn)
print(evaluate_result)

## Traing a DNN Using Plain TensorFlow

#### Construction

In [1]:
import tensorflow as tf
n_inputs = 28 * 28
n_hidden_1 = 256
n_hidden_2 = 100
n_outputs = 10

In [2]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

In [3]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs) # 这个数值可以使得算法更快收束，提高效率
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name='weights')
        b = tf.Variable(tf.zeros([n_neurons]), name='biases')
        z = tf.matmul(X, W) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z

In [ ]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden_1, "hidden1", activation='relu')
    hidden2 = neuron_layer(hidden1, n_hidden_2, "hidden2", activation='relu')
    logits = neuron_layer(hidden2, n_outputs, "outputs")

In [4]:
# 使用tf的默认函数组织dnn
from tensorflow.contrib.layers import fully_connected
with tf.name_scope("dnn_tf"):
    hidden1 = fully_connected(X, n_hidden_1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden_2, scope='hidden2')
    logits = fully_connected(hidden2, n_outputs, scope='outputs', activation_fn=None)

In [5]:
with tf.name_scope("loss"):
    #这个函数，自动计算了神经网络的cost function？？？
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") # 所有training instances的输出结果的平均数！！！

In [6]:
# create optimizer
learning_rate=0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [7]:
with tf.name_scope("eval"):
    # 输出一组bool值，用于计算预测的结果和实际结果的是否相等
    # tf.nn.in_top_k(预测值， 目标索引，采用前n位)
    # ref：https://blog.csdn.net/UESTC_C2_403/article/details/73187915
    correct = tf.nn.in_top_k(logits, y, 1) 
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:
model_path = '/Users/chance/machine_learning/hands_on_machine_learning/tmp/dnn_model_final.ckpt'
n_epochs = 50
batch_size = 50
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size) #支持直接提取batch数据
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                            y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, model_path)

0 Train accuracy: 0.88 Test accuracy: 0.9021
1 Train accuracy: 0.92 Test accuracy: 0.9202
2 Train accuracy: 0.88 Test accuracy: 0.9304
3 Train accuracy: 1.0 Test accuracy: 0.9366
4 Train accuracy: 0.9 Test accuracy: 0.942
5 Train accuracy: 0.92 Test accuracy: 0.9458
6 Train accuracy: 1.0 Test accuracy: 0.9495
7 Train accuracy: 0.92 Test accuracy: 0.952
8 Train accuracy: 0.98 Test accuracy: 0.9564
9 Train accuracy: 1.0 Test accuracy: 0.957
10 Train accuracy: 0.94 Test accuracy: 0.9587
11 Train accuracy: 0.98 Test accuracy: 0.9604
12 Train accuracy: 1.0 Test accuracy: 0.9636
13 Train accuracy: 1.0 Test accuracy: 0.9649
14 Train accuracy: 0.98 Test accuracy: 0.9659
15 Train accuracy: 0.96 Test accuracy: 0.9667
16 Train accuracy: 0.94 Test accuracy: 0.9668
17 Train accuracy: 0.94 Test accuracy: 0.9692
18 Train accuracy: 1.0 Test accuracy: 0.9699
19 Train accuracy: 0.96 Test accuracy: 0.9727
20 Train accuracy: 0.98 Test accuracy: 0.9725
21 Train accuracy: 0.96 Test accuracy: 0.9726
22 Train

In [18]:
# Using the Neural Network
import numpy as np
test_image = mnist.test.images[0]
test_label = mnist.test.labels[0]
with tf.Session() as sess:
    saver.restore(sess, model_path)
    Z = logits.eval(feed_dict={X: [test_image]})
    y_pred = np.argmax(Z, axis=1)
    print("Test predict:", y_pred, "correct:", test_label)

INFO:tensorflow:Restoring parameters from /Users/chance/machine_learning/hands_on_machine_learning/tmp/dnn_model_final.ckpt
Test predict: [7] correct: 7
